# Finetuning Transformer Models

In this tutorial we will finetune a transformer model for a dataset on [emotions](https://huggingface.co/datasets/dair-ai/emotion). In this dataset, there are 6 possible emotions such as sadness (0), joy (1), love (2), anger (3), fear (4) and surprise (5).

# Steps
1. At first, we will load the data and look at its attributes.
2. Then we will tokenize the data.
3. After that, we will set the model parameters and define the model.
4. Then we will set the training arguments.
5. Finally we will train the model and evaluate its performance.

In [1]:
!pip install 'transformers[torch]'
!pip install datasets zstandard evaluate
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00


In [2]:
import datasets
from datasets import DatasetInfo
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
import torch
import evaluate
import numpy as np

In [3]:
# connect to GPU runtime to check this
print(f'(Free memory, Available Memory){torch.cuda.mem_get_info()}')

(Free memory, Available Memory)(15727656960, 15835660288)


In [4]:
# Load Dataset
ds = datasets.load_dataset("dair-ai/emotion")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [25]:
# Dataset Information
DatasetInfo(ds)

DatasetInfo(description=DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
}), citation='', homepage='', license='', features=None, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, dataset_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

In [5]:
ds["train"][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [6]:
# Tokenize dataset
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [8]:
tokenized_ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
# define our evaluation metrics
accuracy = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
# These variables are used to map from character labels to numeric labels. In this case we do not need them because we only have numberic labels.
id2label = {0: 0, 1: 1, 2:2, 3:3, 4:4, 5:5}
label2id = {0: 0, 1: 1, 2:2, 3:3, 4:4, 5:5} #sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5)

In [21]:
# Define the model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=6) #id2label=id2label, label2id=label2id


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# Set training arguments
training_args = TrainingArguments(
    output_dir="my_shiny_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)



In [23]:
# Create an instance of the trainer with the set training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



In [24]:
# Train Model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.254200,0.191512,0.927500
2,0.141300,0.154112,0.938500


Checkpoint destination directory my_shiny_model/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_shiny_model/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=2000, training_loss=0.31453696060180664, metrics={'train_runtime': 181.967, 'train_samples_per_second': 175.856, 'train_steps_per_second': 10.991, 'total_flos': 389287358125632.0, 'train_loss': 0.31453696060180664, 'epoch': 2.0})

In [19]:
# Evaluate Model
trainer.evaluate(tokenized_ds["train"])

{'eval_loss': 0.09362220764160156,
 'eval_accuracy': 0.959375,
 'eval_runtime': 26.8417,
 'eval_samples_per_second': 596.088,
 'eval_steps_per_second': 37.255,
 'epoch': 2.0}

In [20]:
trainer.evaluate(tokenized_ds["validation"])

{'eval_loss': 0.15073446929454803,
 'eval_accuracy': 0.9385,
 'eval_runtime': 3.2226,
 'eval_samples_per_second': 620.613,
 'eval_steps_per_second': 38.788,
 'epoch': 2.0}

In [18]:
trainer.evaluate(tokenized_ds["test"])

{'eval_loss': 0.17803671956062317,
 'eval_accuracy': 0.9245,
 'eval_runtime': 3.0922,
 'eval_samples_per_second': 646.781,
 'eval_steps_per_second': 40.424,
 'epoch': 2.0}

# To do
1. Choose another one or two suitable models from huggingface and compare their performance.

2. Reflect and discuss how you would improve the performance of these models.